## Load Parameters & Other Essentials

### Load the Training Dataset

In [124]:
dataset_file = './trainingdata_stepwise_turkish_3_articulators.tsv'

### Load the packages and functions

In [125]:
from dev import *
import pandas as pd
import numpy as np
import torch
import statsmodels.api as sm

data_stepwise = Dataset(dataset_file)

./trainingdata_stepwise_turkish_3_articulators.tsv


/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most r

## Evaluation Data Preparation

### Define the articulators to be used by different models

In [126]:
rounding = ["la_output", "tb_output"]
round_and_front = ["la_output", "tb_output", "tc_output"]
front = ["tb_output", "tc_output"]
analyses = [rounding, round_and_front, front]

def get_analysis_name(ls: list) -> str:
    if ls == rounding:
        return 'rounding'
    if ls == round_and_front:
        return 'round_and_front'
    if ls == front:
        return 'fronting'

# load dataframe
data = pd.read_csv(dataset_file, sep='\t')
data = data[data['syllables']==2]

### Create several models for each analysis and construct a dataframe

In [127]:
# define functions to be used when building the dataframe

# helper function to get decoder outputs
def get_decoder(input: torch.Tensor, target: torch.Tensor, model) -> np.ndarray:
    with torch.no_grad():
        _, attn_map_seq = model(input, target)
    return attn_map_seq.numpy()[:,0] # attention paid to the first letter

# helper functions to get correct inputs
def get_trial(training_data, word, model):
    trial = training_data.make_trial(word)
    return trial[0], torch.cat((trial[1], trial[2]), axis=1), model

# get the decoder outputs for each word
get_out = lambda x, y, model : pd.DataFrame(get_decoder(*get_trial(x, y, model)))

# iterate through each analysis
df_all = None
for k, analysis in enumerate(analyses):
    # iterate over 20 different models
    for j in range(20):
        # train a new model
        model = Seq2Seq(training_data=data_stepwise, articulators=analysis)
        model.train_model(training_data=data_stepwise, n_epochs=200)
        # model.save()

        # get the attention values from the model
        df = get_out(data_stepwise, data['underlying'].values[0], model).T
        for i in range(1, data['underlying'].shape[0]):
            df = pd.concat(
                (df, get_out(data_stepwise, data['underlying'].values[i], model).T),
                axis=0
            )

        # reset the indexes
        df = df.reset_index().drop('index', axis=1)

        # add columns
        for c in ['underlying', 'consonant', 'vowel']:
            col = data[c]
            col = col.reset_index().drop('index', axis=1)
            df[c] = col

        df = df.rename({'vowel': "V2"}, axis=1)
        df = df.assign(
            V1 = lambda d: d['underlying'].astype(str).str[0]
        )

        df = df.assign(
            group = get_analysis_name(analysis)
        )

        df = df.assign(
            model = (j+1) + (k * 20)
        )
    
        if isinstance(df_all, pd.DataFrame):
            df_all = pd.concat((df_all, df), axis=0)
        else:
            df_all = df

        print(end='\x1b[2K')
        print(f"{(j+1) + (k * 20)}/{20 * len(analyses)}")

df_all = df_all.reset_index().drop('index', axis=1)
print(df_all)

100%|██████████| 200/200 [02:38<00:00,  1.26it/s]


1/60


100%|██████████| 200/200 [02:38<00:00,  1.26it/s]


2/60


100%|██████████| 200/200 [02:40<00:00,  1.25it/s]


...

58/60


100%|██████████| 200/200 [02:37<00:00,  1.27it/s]


59/60


100%|██████████| 200/200 [02:42<00:00,  1.23it/s]

60/60
             0         1         2         3         4         5         6  \
0     0.877493  0.891837  0.891027  0.837420  0.134618  0.484158  0.308495   
1     0.940751  0.972527  0.976617  0.866025  0.120465  0.509531  0.234733   
2     0.940558  0.957797  0.959046  0.919910  0.181786  0.547717  0.326072   
3     0.523138  0.526685  0.527616  0.500620  0.306883  0.288392  0.228648   
4     0.545067  0.561122  0.564030  0.545583  0.358564  0.297703  0.257977   
...        ...       ...       ...       ...       ...       ...       ...   
2875  0.491988  0.091367  0.059237  0.053706  0.032351  0.015354  0.014646   
2876  0.482587  0.101485  0.073986  0.065135  0.037305  0.011806  0.016383   
2877  0.345552  0.076055  0.048850  0.041997  0.036490  0.033191  0.031915   
2878  0.341192  0.036247  0.021226  0.019891  0.020082  0.021487  0.018615   
2879  0.444391  0.093248  0.070856  0.061836  0.034950  0.010113  0.016762   

             7         8         9 underlying consonant V

### Prep dataframe for analysis

In [128]:
# create additional categorical values
df_a = df_all.assign(
    rounded = lambda d: d["V1"].apply(lambda y: 1 if y in ["ø", "u", "y", "o"] else 0)
)
df_b = df_a.assign(
    fronted = lambda d: d["V1"].apply(lambda y: 1 if y in ["ø", "e", "y", "i"] else 0)
)
df_c = df_b.assign(
    high = lambda d: d["V1"].apply(lambda y: 1 if y in["ø", "u", "y", "o", "i", "ɯ"] else 0)
)
print(df_c)

             0         1         2         3         4         5         6  \
0     0.877493  0.891837  0.891027  0.837420  0.134618  0.484158  0.308495   
1     0.940751  0.972527  0.976617  0.866025  0.120465  0.509531  0.234733   
2     0.940558  0.957797  0.959046  0.919910  0.181786  0.547717  0.326072   
3     0.523138  0.526685  0.527616  0.500620  0.306883  0.288392  0.228648   
4     0.545067  0.561122  0.564030  0.545583  0.358564  0.297703  0.257977   
...        ...       ...       ...       ...       ...       ...       ...   
2875  0.491988  0.091367  0.059237  0.053706  0.032351  0.015354  0.014646   
2876  0.482587  0.101485  0.073986  0.065135  0.037305  0.011806  0.016383   
2877  0.345552  0.076055  0.048850  0.041997  0.036490  0.033191  0.031915   
2878  0.341192  0.036247  0.021226  0.019891  0.020082  0.021487  0.018615   
2879  0.444391  0.093248  0.070856  0.061836  0.034950  0.010113  0.016762   

             7         8         9 underlying consonant V2 V1  

In [129]:
df_melt = pd.melt(
    frame=df_c,
    id_vars=[
        "V1", "V2", "consonant", "underlying", "fronted", "rounded", "high", "group", "model"
    ],
    value_name="Attention",
    value_vars=[5, 6, 7, 8, 9],
    var_name="Time"
)

# set the categories as well
df_mle = df_melt.astype(
    {
        "Time": 'int64', 
        "V1": 'category', 
        "V2": 'category', 
        "consonant": 'category', 
        "fronted": 'category', 
        "rounded": 'category', 
        "high": 'category', 
        "underlying": 'category',
        "group": 'category',
        "model": 'category'
    }
)
print(df_mle)

      V1 V2 consonant underlying fronted rounded high     group model  Time  \
0      i  H         b       ib-H       1       0    1  rounding     1     5   
1      a  H         b       ab-H       0       0    0  rounding     1     5   
2      e  H         b       eb-H       1       0    0  rounding     1     5   
3      o  H         b       ob-H       0       1    1  rounding     1     5   
4      u  H         b       ub-H       0       1    1  rounding     1     5   
...   .. ..       ...        ...     ...     ...  ...       ...   ...   ...   
14395  o  L         d       od-L       0       1    1  fronting    60     9   
14396  u  L         d       ud-L       0       1    1  fronting    60     9   
14397  y  L         d       yd-L       1       1    1  fronting    60     9   
14398  ø  L         d       ød-L       1       1    1  fronting    60     9   
14399  ɯ  L         d       ɯd-L       0       0    1  fronting    60     9   

       Attention  
0       0.484158  
1       0.509

## Evaluate groups & models

In [130]:
# data = sm.datasets.get_rdataset("dietox", "geepack").data
# print(data.dtypes)

# save the dataframe
df_mle.to_csv('model_outputs.csv')

### Descriptives

In [131]:
# mean attention for each group for each V2
df_mle.groupby(["group", "V2"])[["Attention"]].describe()

Attention                                          \
                       count      mean       std       min       25%   
group           V2                                                     
fronting        H     2400.0  0.123066  0.143504  0.000032  0.030641   
                L     2400.0  0.135016  0.144760  0.000025  0.038377   
round_and_front H     2400.0  0.367989  0.350564  0.000007  0.050117   
                L     2400.0  0.339823  0.350569  0.000010  0.033970   
rounding        H     2400.0  0.417634  0.271103  0.000390  0.165114   
                L     2400.0  0.331111  0.297902  0.000014  0.064440   

                                                  
                         50%       75%       max  
group           V2                                
fronting        H   0.078244  0.160451  1.244949  
                L   0.096677  0.179309  1.279259  
round_and_front H   0.253214  0.654266  1.510183  
                L   0.194170  0.620681  1.515658  
rounding        H   0.428148  0.617737  1.236852  
                L   0.235404  0.563806  1.247287

### Mixed linear effect models for each group

In [132]:
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import MultiComparison

''' From paper:
We used the identity of input V2 as 
    - either a high harmony trigger (/i, u/) 
      or a non-high non-trigger
    - and decoder timepoint as main factors, 
    - model as a random factor, 
    - and the attention value assigned to the encoder hidden state 
        associated with input V2 as the dependent variable.

This result suggests that the
decoder learns to pay more attention to a V2 at an
earlier timepoint when that V2 is a harmony trigger,
consistent with the representation of an anticipatory
(early-activating) gesture assumed by the Gestural
Harmony Model.
'''

# first fit a mixed linear model for each group

# divide the df into groups
rounding_df = df_mle[df_mle['group'] == 'rounding']
fronting_df = df_mle[df_mle['group'] == 'fronting']
two_way_df = df_mle[df_mle['group'] == 'round_and_front']
grouped_dfs = [rounding_df, fronting_df, two_way_df]

# perform an MLE on each group
linear_models = []
for grp in grouped_dfs:
  # model training (see: https://stats.stackexchange.com/questions/415041/am-i-using-the-right-linear-mixed-model-design-for-my-data)
  md = smf.mixedlm("Attention ~ Time + V2 + fronted + rounded + high", grp, groups=grp["model"])
  mdf = md.fit(reml=False)
  linear_models.append(mdf)
  print(f"{grp['group'].values[0]} results:")
  print(mdf.summary())

  # # tukey HSD
  mc = MultiComparison(grp['Attention'], groups=grp['V2'])
  print(mc.tukeyhsd().summary())

/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


rounding results:
         Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: Attention
No. Observations:  4800    Method:             ML       
No. Groups:        20      Scale:              0.0660   
Min. group size:   240     Log-Likelihood:     -318.2120
Max. group size:   240     Converged:          Yes      
Mean group size:   240.0                                
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept    -0.104    0.027  -3.845 0.000 -0.158 -0.051
V2[T.L]      -0.087    0.007 -11.669 0.000 -0.101 -0.072
fronted[T.1]  0.045    0.007   6.004 0.000  0.030  0.059
rounded[T.1]  0.072    0.009   7.942 0.000  0.054  0.090
high[T.1]     0.020    0.010   1.949 0.051 -0.000  0.041
Time          0.064    0.003  24.437 0.000  0.059  0.069
Group Var     0.006    0.012                            

Multiple Comparison of

/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
     H      L   0.0119 0.0041 0.0038 0.0201   True
--------------------------------------------------
round_and_front results:
         Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: Attention
No. Observations:  4800    Method:             ML       
No. Groups:        20      Scale:              0.0560   
Min. group size:   240     Log-Likelihood:     73.3934  
Max. group size:   240     Converged:          Yes      
Mean group size:   240.0                                
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept    -0.842    0.027 -30.856 0.000 -0.895 -0.788
V2[T.L]      -0.028    0.007  -4.125 0.000 -0.042 -0.015
fronted[T.1]  0.046    0.007   6.776 0.000  0

/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
     H      L  -0.0282 0.0054 -0.048 -0.0083   True
---------------------------------------------------


### Compare BIC

In [133]:
# compare model BIC
for m, grp in zip(linear_models, grouped_dfs):
    print(f"{grp['group'].values[0]} BIC: {m.bic}")

rounding BIC: 704.2350027424596
fronting BIC: -5964.747841267892
round_and_front BIC: -78.97573248954939


### Compare attention across groups (one model)

In [134]:
# one mixed linear effects model
md = smf.mixedlm("Attention ~ Time + group * V2", df_mle, groups=df_mle["model"])
mdf = md.fit()
print(mdf.summary())

## tukey HSD
mc = MultiComparison(df_mle['Attention'], groups=df_mle['group'])
print(mc.tukeyhsd().summary())

/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                   Mixed Linear Model Regression Results
Model:                   MixedLM        Dependent Variable:        Attention
No. Observations:        14400          Method:                    REML     
No. Groups:              60             Scale:                     0.0563   
Min. group size:         240            Log-Likelihood:            165.3222 
Max. group size:         240            Converged:                 Yes      
Mean group size:         240.0                                              
----------------------------------------------------------------------------
                                 Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                        -0.448    0.020 -22.416 0.000 -0.487 -0.409
group[T.round_and_front]          0.245    0.025   9.931 0.000  0.197  0.293
group[T.rounding]                 0.295    0.025  11.944 0.000  0.246  0.343
V2[T.L]            